#### Imports

In [29]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [30]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory,'..','..','oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex = hell

In [31]:
regex = r'SAW_06_02_2024_OIL_TRAFO_38_(\d+)\.S2P'

In [32]:
def read_s2p_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "TRAFO 38"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência e parâmetros S do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db
            
            #Dataframe com informações do objeto Network
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S11 (db)' : s_parameters[:,0,0],
                'S21 (db)' : s_parameters[:,1,0],
                'S12 (db)' : s_parameters[:,0,1],
                'S22 (db)' : s_parameters[:,1,1],
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [33]:
df_mineral = read_s2p_data(regex, s2ps_path)

In [34]:
df_mineral.info

<bound method DataFrame.info of        OIL NAME  MEASUREMENT NUMBER  FREQUENCY (MHZ)  S11 (db)  S21 (db)  \
0      TRAFO 38                   1       110.000000 -0.969863 -82.68540   
1      TRAFO 38                   1       110.002000 -0.970456 -73.82391   
2      TRAFO 38                   1       110.004000 -0.971064 -78.46970   
3      TRAFO 38                   1       110.006001 -0.971451 -80.12454   
4      TRAFO 38                   1       110.008001 -0.970726 -76.48322   
...         ...                 ...              ...       ...       ...   
29995  TRAFO 38                   3       129.991999 -1.372363 -70.82593   
29996  TRAFO 38                   3       129.993999 -1.372485 -68.65086   
29997  TRAFO 38                   3       129.996000 -1.371827 -69.72959   
29998  TRAFO 38                   3       129.998000 -1.371638 -67.85474   
29999  TRAFO 38                   3       130.000000 -1.372850 -67.81706   

       S12 (db)  S22 (db)  
0     -79.72281 -1.140833  

In [35]:
#assegura somente óleo de girassol lido
for name in df_mineral["OIL NAME"]:
    if name != "TRAFO 38":
        print("Diferente de TRAFO 38")

In [36]:
#cria .csv caso precise no futuro.
path_save_csv = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df_mineral.to_csv(path_save_csv + '/TRAFO_38_measurements.csv')

In [37]:
def average_measurements(dataframe):
    #agrupa valores por frequência e calcula média do que interessa
    return dataframe.groupby('FREQUENCY (MHZ)', as_index=False)['S21 (db)'].mean()

In [38]:
df_TRAFO_38_s21 = average_measurements(df_mineral)

In [39]:
df_TRAFO_38_s21.info


<bound method DataFrame.info of       FREQUENCY (MHZ)   S21 (db)
0          110.000000 -83.183533
1          110.002000 -78.368323
2          110.004000 -77.650627
3          110.006001 -79.275413
4          110.008001 -76.703070
...               ...        ...
9995       129.991999 -70.272337
9996       129.993999 -68.959770
9997       129.996000 -68.903420
9998       129.998000 -68.276770
9999       130.000000 -67.847340

[10000 rows x 2 columns]>

In [40]:
#cria .csv caso precise no futuro.
path_save_csv_processed = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs','processed')
if not os.path.exists(path_save_csv_processed):
   os.makedirs(path_save_csv_processed)
df_TRAFO_38_s21.to_csv(path_save_csv_processed + '/TRAFO_38_measurements_processed.csv')